In [2]:
import random,sys,math
import numpy as np
from itertools import permutations

In [3]:
#fonction pour calculer la distance d'un individu dans la population
def calculateDist(child,distances):
    d=0
    for i in range(0,len(child)-1):
        d+=distances[int(child[i])][int(child[i+1])]

    d+=distances[int(child[len(child)-1])][int(child[0])]
    return d


#la fonction du 'Order crossOver'
def croissement(population,distances):
    posParent1=0
    posParent2=0
    
    #defini la pourcentage de la croissement de 20% de la taille du population
    sizeOfcross=math.ceil(len(population)*0.2)
    
    #on repete la selection dns le cas ou on a selectionne
    while posParent1==posParent2:
        #choisir aléatoirement les position du parent1 et parent2 dans la population
        posParent1=random.randint(0,len(population)-1)
        posParent2=random.randint(0,len(population)-1)
        
    #extract et garde les parent1 et parent2 dans des variables parent1,parent2   
    parent1=population[posParent1]
    parent2=population[posParent2]
    
    #initialisation du tableau de l'enfant avec des valeurs de -1
    child=np.ones(len(parent1))*-1
    
    #choisir au hasard la position de départ du crossover
    startPos=random.randint(1,len(parent1))
    
    #prendre 20% du nombre de valeurs de parent1 à partir du startPos.
    for i in range(startPos,(startPos+sizeOfcross)):
        try:
            child[i]=int(parent1[i])
    #dans le cas ou il depasse la longueur de l'enfant et du parent1, il continue les itérations depuis le debut des tableaux
        except Exception as e:
            child[len(parent1)-i]=parent1[len(parent1)-i]
    i=0
    j=0
    
    #remplir les restes des valeurs de l'enfant par les valeur du parent2 
    while 1:
        #si cette position est deja rempli avec une valeur du parent1, passez à la position suivante
        if child[i]!=-1:
            i+=1
            #si le compteur i dépasse la longueur de parent2 terminer le processus de remplissage
            if i>len(parent2)-1:
                break
            continue
            
        #si la valeur du parent2 est déjà dans l'enfant, passez à l'itération suivante
        if parent2[j] in child:
            j+=1
            if j>len(parent2)-1:
                break
            continue
        
        #sinon remplir la valeur de parent2 dans l'enfant
        child[i]=parent2[j]
        j+=1
        i+=1
        
        #si nous dépassons la longueur de l'enfant ou du parent, le processus de remplissage est termine
        if j>len(parent2)-1 or i>len(child)-1:
            break
            
    #si dans le cas ou le nouvel enfant cree a un element identique dans la population, nous renvoyons un tableau vide
    for elt in population:
        if np.array_equal(child,elt):
            return ([],-1)
        
    #sinon on retourne le nouvel enfant avec sa distance calculée par la fonction calculateDist
    distance=calculateDist(child,distances)
    return (child,distance)
        
#la focntion du 'swap mutation'              
def swapMutation(population,pathCost,distances):
    pos1=0
    pos2=0
    
    #choisir au hasard l'individu qui aura la mutation
    mutationPos=random.randint(0,len(population)-1)
    member=population[mutationPos]
    
    while 1:
        #choisissez au hasard les positions pos1 et pos2 pour les permuter
        pos1=random.randint(0,len(member)-1)
        pos2=random.randint(0,len(member)-1)
        
        #si pos1 et pos2 sont identiques,repeter le processus jusqu'a ce que nous obtenions deux positions differentes
        if pos1!=pos2:
            break
    #appliquer le processus d'échange entre les valeurs de pos1 et pos2
    tmp=member[pos1]
    member[pos1]=member[pos2]
    member[pos2]=tmp
    
    #recalculer la nouvelle distance apres la mutation
    cost=calculateDist(member,distances)
    
    #remplacer la distance du membre choisi par la nouvelle distance du membre mute
    pathCost[mutationPos]=cost
    
    #remplacer le membre choisi par le nouveau membre mute
    population[mutationPos]=member
    
#la fonction du 'Tournament Selection'     
def selection(population,pathCost):
    chosenParent = None
    
    #si le nombre de la moitie de la population est inférieur à 2,nous enregistrons le longueur de la population dans k
    if(int(len(population)/2))<2:
        k=len(population)
        
    #sinon on prend un nombre aleatoire entre 2 et (le nombre d'elements restants a choisir)/2
    else:
        k=random.randint(2,int(len(population)/2))
        
    contenders=[]
    min=1000000
    
    for i in range(0,k):
        #si(le nombre d'elemets de la population)/2 inferieur à 2,remplir tous les membres dans le tableau contenders avec leur dsitance
        if(int(len(population)/2))<2:
            contenders.append((population[i],pathCost[i]))
            continue
        #sinon on choisit k element aleatoire dans cette population
        while 1:
            #choisissez un element aleatoire avec sa distance calculee jusqu'a ce que nous trouvions un element qui n'est pas dans le tableau 'contenders'
            parentPos=random.randint(2,len(population)-1)
            if (population[parentPos],pathCost[parentPos]) not in contenders:
                break
        #enregistrer le tuple contenant l'element choisi avec sa distance calculee dans le tableau des pretendants     
        contenders.append((population[parentPos],pathCost[parentPos]))
        
    #trouver l'element avec la distance minimale dans le tableau 'contenders'
    for parent in contenders:
        if parent[1]<min:
            chosenParent=parent
            
    #renvoie le parent avec la distance minimale et le tableau contenders
    return chosenParent,contenders

        
        
    
    

In [ ]:
#choisir un nombre au hasard pour le nombre de villes
nbCities=random.randint(10,20)
print(nbCities-1)

#initialiser le tableau des villes et le tableau des distances entre chaque deux villes
cities=[]
distances=np.zeros((nbCities-1,nbCities-1))
for i in range(0,nbCities-1):
    cities.append(i)

#remplir le tableau 'distances' avec des distances aléatoires entre 2 villes d'une manière à obtenir une matrice d'ajacance
for i in range(0,nbCities-1):
    for j in range(0,nbCities-1):
        if i==j:
            distances[i][j]=0
            distances[j][i]=distances[i][j]
            continue
        distances[i][j]=random.randint(10,20)
        distances[j][i]=distances[i][j]
#print(distances,'\n')


pathCost=[]

#generer des chemins aleatoires pour le tableau de population pour démarrer l'algorithme avec ces valeurs
population = [random.sample(cities, nbCities-1) for i in range(len(cities))]

#calculer les distances de ces chemins initials et les guarder dans le tableau pathCost
for i in range(0,len(population)):
    cost=0
    for j in range(0,len(population[i])-1):
        cost+=distances[population[i][j]][population[i][j+1]]
    cost+=distances[population[i][len(population)-1]][population[i][0]]
    pathCost.append(cost)

#demarer l'algorithm genetique avec 200 iterations
for i in range(0,200):
    
    #s'il ne reste qu'un seul chemin dans le tableau 'population', terminez l'algorithme
    if(len(population)==1):
        break
    #applique 3 croisement
    for j in range(0,3):
        (newBorn,newdistance)=croissement(population,distances)
        #si la fonction renvoie un tableau vide et une distance de -1, cela signifie que l'enfant existe déjà
        #donc passez à l'iteration suivante
        if len(newBorn)==0 and newdistance==-1:
             continue
                
        # sinon enregistrer le nouveau-né dans le tableau 'population' et sa distance dans le tableau 'pathCost'
        population.append(list(newBorn))
        pathCost.append(newdistance)
    #applique 5 mutations    
    for j in range(0,5):
         swapMutation(population,pathCost,distances)
    #applique 2 fois la selection
    for j in range(0,2):
        (chosenParent,contenders)=selection(population,pathCost)
        
        #utilisez le tableau 'contenders' pour supprimer du tableau 'population' 
        #les elements qui n'ont pas ete choisis dans la fonction selection
        for contender in contenders:
            
            #si cet element du tableau 'contenders' est l'element choisi par la fonction de selection
            #nous passons a l'iteration suivante sans le supprimer du tableau 'population'
            if np.array_equal(contender[0],chosenParent[0]):
                continue
                
            #supprimer l'element du tableau 'population'
            population.remove(contender[0])
        #recalculer le distance des elements de population car leur position dans le tableau a changer apres la supprimation des elements
        pathCost=[]
        for i in range(0,len(population)):
            cost=0
            for j in range(0,len(population[i])-1):
                cost+=distances[int(population[i][j])][int(population[i][j+1])]
            cost+=distances[int(population[i][len(population[i])-1])][int(population[i][0])]
            pathCost.append(cost)
                        
print("")
print("")
print("")
print("")

#affichage du resultat final
print("final result:",population)
print("the distance:",pathCost)


    
    
    